In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

In [26]:
data = pd.read_csv("C:/Users/Aryaman/Downloads/Bengaluru_House_Data.csv")
data

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


In [27]:
data=data.drop(['area_type','availability','society'],axis=1)
data.shape

(13320, 6)

In [28]:
data.isnull().sum()

location        1
size           16
total_sqft      0
bath           73
balcony       609
price           0
dtype: int64

In [29]:
data=data.drop(['balcony'],axis=1)

In [30]:
data.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [31]:
data['size']=data['size'].str.split(' ').str.get(0)

In [32]:
data['location'].fillna(data['location'].mode()[0],inplace=True)

In [35]:
data['size']=data['size'].str.split(' ').str.get(0)

In [39]:
data['size'].fillna(data['size'].median(),inplace=True)

In [ ]:
data['size']=data['size'].astype(int)

In [45]:
data['bath'].fillna(round(data['bath'].mean()),inplace=True)

In [48]:
data['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

In [49]:
# def isFloat(x):
#     try:
#         float(x)
#     except:
#         return False
#     return True
def conv_sqft_to_num(x):
    tokens = x.split('-')
    if(len(tokens)==2):
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [53]:
data['total_sqft']=data['total_sqft'].apply(conv_sqft_to_num)

In [54]:
data['total_sqft'].unique()

array([1056. , 2600. , 1440. , ..., 1258.5,  774. , 4689. ])

In [55]:
data['price_per_sqft']=(data['price']*100000)/data['total_sqft']

In [61]:
data['location']=data['location'].apply(lambda x: x.strip())
location_count = data['location'].value_counts()

In [62]:
location_count_less_than_10 = location_count[location_count<=10]
location_count_less_than_10

Dairy Circle                      10
Nagappa Reddy Layout              10
Basapura                          10
1st Block Koramangala             10
Sector 1 HSR Layout               10
                                  ..
Bapuji Layout                      1
1st Stage Radha Krishna Layout     1
BEML Layout 5th stage              1
singapura paradise                 1
Abshot Layout                      1
Name: location, Length: 1053, dtype: int64

In [63]:
data['location']=data['location'].apply(lambda x: 'others' if x in location_count_less_than_10 else x)

In [68]:
data=data[((data['total_sqft']/data['size'])>=200)]

In [69]:
data.describe()

,size,total_sqft,bath,price,price_per_sqft
count,12934.000000,12934.00000,12934.000000,12934.000000,12934.000000
mean,2.711304,1578.14456,2.614118,112.090425,6491.260043
std,1.072491,1248.09095,1.159147,150.497762,4338.535153
min,1.000000,250.00000,1.000000,8.000000,267.829813
25%,2.000000,1108.00000,2.000000,49.802500,4239.458956
50%,3.000000,1293.00000,2.000000,70.000000,5356.496640
75%,3.000000,1693.00000,3.000000,120.000000,7090.909091
max,27.000000,52272.00000,27.000000,3600.000000,176470.588235


In [70]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        gen_df = subdf[(subdf.price_per_sqft > (m-st)) & (subdf.price_per_sqft <= (m+st))]
        df_output = pd.concat([df_output,gen_df],ignore_index=True)
    return df_output
data = remove_outliers_sqft(data)
data.describe()

,size,total_sqft,bath,price,price_per_sqft
count,10706.000000,10706.000000,10706.000000,10706.000000,10706.000000
mean,2.613208,1501.741482,2.508967,91.888422,5751.356998
std,0.980772,880.291289,1.053614,86.142905,2396.624824
min,1.000000,284.000000,1.000000,8.000000,1379.310345
25%,2.000000,1101.000000,2.000000,49.000000,4252.311610
50%,2.000000,1280.000000,2.000000,68.000000,5208.333333
75%,3.000000,1650.000000,3.000000,100.000000,6528.370958
max,27.000000,30400.000000,27.000000,2200.000000,24509.803922


In [77]:
def bhk_outlier_remover(df):
    exclude_indices = np.array([])
    for location,location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk,bhk_df in location_df.groupby('size'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk,bhk_df in location_df.groupby('size'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [79]:
data=bhk_outlier_remover(data)
data

,location,size,total_sqft,bath,price,price_per_sqft
0,1st Block Jayanagar,4,2850.0,4.0,428.0,15017.543860
1,1st Block Jayanagar,3,1630.0,3.0,194.0,11901.840491
2,1st Block Jayanagar,6,1200.0,6.0,125.0,10416.666667
3,1st Block Jayanagar,3,1875.0,2.0,235.0,12533.333333
4,1st Block Jayanagar,3,1200.0,2.0,130.0,10833.333333
...,...,...,...,...,...,...
10696,others,2,1200.0,2.0,70.0,5833.333333
10697,others,1,1800.0,1.0,200.0,11111.111111
10701,others,2,1353.0,2.0,110.0,8130.081301
10702,others,1,812.0,1.0,26.0,3201.970443


In [80]:
data.drop(columns='price_per_sqft',inplace=True)

In [82]:
data.to_csv('cleaned_data.csv')

In [86]:
X = data.drop(['price'],axis=1)
Y = data['price']

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline 
from sklearn.metrics import r2_score

In [88]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

In [91]:
ohe = OneHotEncoder()
ohe.fit(X[['location']])

OneHotEncoder()

In [92]:
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['location']),remainder='passthrough')

In [93]:
lr = LinearRegression()

In [94]:
pipe = make_pipeline(column_trans,lr)

In [95]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['1st Block Jayanagar', '1st Phase JP Nagar',
       '2nd Phase Judicial Layout', '2nd Stage Nagarbhavi',
       '5th Block Hbr Layout', '5th Phase JP Nagar', '6th Phase JP Nagar',
       '7th Phase JP Nagar', '8th Phase JP Nagar', '9th Phase JP Nagar',
       'AECS Layout',...
       'Thubarahalli', 'Thyagaraja Nagar', 'Tindlu', 'Tumkur Road',
       'Ulsoor', 'Uttarahalli', 'Varthur', 'Varthur Road', 'Vasanthapura',
       'Vidyaranyapura', 'Vijayanagar', 'Vishveshwarya Layout',
       'Vishwapriya Layout', 'Vittasandra', 'Whitefield',
       'Yelachenahalli', 'Yelahanka', 'Yelahanka New Town', 'Yelenahalli',
       'Yeshwanthpur', 'others'], dtype=object)]),
                                                  ['location'])])),
               

In [96]:
y_test_pred = pipe.predict(x_test)

In [97]:
testing_data_accuracy = r2_score(y_test,y_test_pred)
print(testing_data_accuracy)

0.8014701417649626


In [98]:
scores = []
for i in range(1000):
    x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=i)
    lr = LinearRegression()
    pipe = make_pipeline(column_trans,lr)
    pipe.fit(x_train,y_train)
    y_test_pred = pipe.predict(x_test)
    scores.append(r2_score(y_test,y_test_pred))

In [99]:
scores[np.argmax(scores)]

0.8981954974635917

In [100]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=np.argmax(scores))
lr = LinearRegression()
pipe = make_pipeline(column_trans,lr)
pipe.fit(x_train,y_train)
y_test_pred = pipe.predict(x_test)
r2_score(y_test,y_test_pred)

0.8981954974635917

In [101]:
import pickle

In [102]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [109]:
pipe.predict(pd.DataFrame([['Indira Nagar',2,1000,2]],columns=['location','size','total_sqft','bath']))

array([175.46704928])

In [106]:
data[data['location']=='Whitefield']

,location,size,total_sqft,bath,price
7491,Whitefield,2,1459.0,2.0,94.82
7496,Whitefield,2,1280.0,2.0,75.00
7498,Whitefield,3,1910.0,3.0,161.00
7499,Whitefield,4,3252.0,4.0,230.00
7503,Whitefield,2,1150.0,2.0,70.00
...,...,...,...,...,...
7961,Whitefield,3,1564.0,3.0,103.00
7966,Whitefield,3,1760.0,3.0,139.00
7967,Whitefield,1,613.0,1.0,48.00
7968,Whitefield,3,1730.0,3.0,125.00
